In [39]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import tensorflow_text as text
import tensorflow_hub as hub
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers


import platform
import time
import pathlib
import os

In [10]:
recipes=pd.read_csv('newresults.csv')

In [11]:
recipes.dropna(inplace=True)
recipes

,title,ingredients,recipe,category
0,Traditional Osso Buco,"2 pounds veal shanks, cut into short lengths ,...",Dust the veal shanks lightly with flour. Melt ...,Main Dishes
1,Italian Sausage Soup with Tortellini,"1 pound sweet Italian sausage, casings removed...","In a 5-quart Dutch oven, brown sausage. Remove...",Vegetable Soup Recipes
2,"Italian Sausage, Peppers, and Onions","6 (4 ounce) links sweet Italian sausage ,2 tab...",Place the sausage in a large skillet over medi...,Sausage
3,Tender Italian Baked Chicken,"¾ cup mayonnaise ,½ cup grated Parmesan cheese...",Preheat oven to 425 degrees F (220 degrees C)....,Italian
4,Easy Pizza Sauce I,"1 (6 ounce) can tomato paste ,1 ½ cups water ,...","Mix together the tomato paste, water, and oliv...",Pizza Sauce Recipes
...,...,...,...,...
10806,Vegetarian Pad Thai,"1 (6.75 ounce) package thin rice noodles ,2 ta...",Place noodles in a heatproof bowl and cover wi...,Thai
10807,Quick and Easy Thai Style Coleslaw,"1 (16 ounce) package shredded coleslaw mix ,¼ ...",Pour the coleslaw mix into a large bowl. Pour ...,No Mayo
10808,Sukhothai Pad Thai,"½ cup white sugar ,½ cup distilled white vineg...",To prepare Pad Thai sauce: In a medium saucepa...,Thai
10809,Slow Cook Thai Chicken,"6 skinless, boneless chicken breast halves - c...","Place the chicken breast strips, bell pepper a...",Thai


In [12]:
recipes.iloc[76
            ]

title                                               Tahini Salad
ingredients    2 pickle-size Persian cucumbers, diced ,1 larg...
recipe         Combine the cucumber, tomato, and radish in a ...
category                                  Cucumber Salad Recipes
Name: 77, dtype: object

In [13]:
title_marker='Æ Title: \n'
ingredients_marker='Ingredients: \n'
instructions_marker='Instructions: \n'

def df_to_string(recipe):
    title=recipe['title'] + '\n\n'
    ingredients=recipe['ingredients']
    instructions=recipe['recipe']
    instructions=instructions.replace('\n', '')
    
    #seperate the ingredients by comma
    ingredients=ingredients.split(',')
    instructions=instructions.split('.')
    
    #add the title marker and the title of the recipe to the return string
    return_string=title_marker + title +ingredients_marker
    
    for ingredient in ingredients:
        return_string += f'▪︎ {ingredient}\n'
        
    return_string+='\n\n'+instructions_marker
    
    for instruction in instructions:
        return_string+=f'▪︎ {instruction}.\n'
    return return_string
        
        
    

In [14]:
recipes_string=''
for i in range(recipes['title'].size):
    recipes_string+=(df_to_string(recipes.iloc[i]))



In [24]:
print(len(recipes_string))

12132467


In [31]:
def split(recipes_string, max_len, step_size = 1):

    lines = []
    next_chars = []

    for i in range(0, len(recipes_string) - max_len, step_size):
        lines.append(recipes_string[i:i+max_len])
        next_chars.append(recipes_string[i+max_len])
    
    return lines, next_chars

In [32]:
max_len = 20

lines, next_chars =  split(recipes_string, max_len = max_len, step_size = 5)
for i in range(10, 15):
    print(lines[i] + "     =>    " + next_chars[i])

2 pounds veal shanks     =>    

nds veal shanks
▪︎       =>    c
eal shanks
▪︎  cut i     =>    n
hanks
▪︎  cut into s     =>    h

▪︎  cut into short      =>    l


In [34]:
chars = sorted(set(recipes_string))
char_indices = {char : chars.index(char) for char in chars}
X = np.zeros((len(lines), max_len, len(chars)))
y = np.zeros((len(lines), 1), dtype = np.int32)
for i, line in enumerate(lines):
	for t, char in enumerate(line):
		X[i, t, char_indices[char]] = 1
	y[i] = char_indices[next_chars[i]]

In [35]:
X.shape, y.shape

((2426490, 20, 110), (2426490, 1))

In [36]:
X[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
train_len = int(0.7*X.shape[0])
X_train = X[0:train_len]
X_val = X[train_len:]

y_train = y[0:train_len]
y_val  = y[train_len:]

In [40]:
model = tf.keras.models.Sequential([
    layers.LSTM(128, name = "LSTM", input_shape=(max_len, len(chars))),
    layers.Dense(len(chars))        
])

In [ ]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
              optimizer = "adam")

In [ ]:
model.fit(X_train, 
          y_train,
          validation_data= (X_val, y_val),
          batch_size=128, epochs = 1)